In [6]:
%load_ext autoreload
%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:

import sys
if '../working' not in sys.path: 
    sys.path.append('../working')

!sage -preparse ../working/normalizer.sage
!mv ../working/normalizer.sage.py ../working/normalizer.py

!sage -preparse ../working/self_similar.sage
!mv ../working/self_similar.sage.py ../working/self_similar.py

!sage -preparse ../working/planar_groups.sage
!mv ../working/planar_groups.sage.py ../working/planar_groups.py


from normalizer import *
from self_similar import *
from planar_groups import *
prepare_gap_env()

In [15]:
SR_Degrees(7).algorithm()

=========================== Group 7 ==================================
Generators of group:

[ [-1  0  0]  [ -1   0 1/2]  [1 0 1]  [1 0 0] ]
[ [ 0 -1  0]  [  0   1   0]  [0 1 0]  [0 1 1] ]
[ [ 0  0  1], [  0   0   1], [0 0 1], [0 0 1] ]

SNoT

[ [-1  0| 0]  [ -1   0|1/2]  [1 0|0]  [   1    0|-1/2] ]
[ [ 0 -1| 0]  [  0   1|  0]  [0 1|0]  [   0   -1|   0] ]
[ [-----+--]  [-------+---]  [---+-]  [---------+----] ]
[ [ 0  0| 1], [  0   0|  1], [0 0|1], [   0    0|   1] ]


[ [x0  0]  [ 0 x1] ]
[ [ 0 x1], [x0  0] ]


----------------Dilation-------------------------

... testing inverse A (should have integral entities):

[x0  0]
[ 0 x1]

      [-1  0| 0]                       [-1  0| 0]
      [ 0 -1| 0]    [  -1    0 2*a0]   [ 0 -1| 0]
a_inv [-----+--]a = [   0   -1 2*a1] = [-----+--]
      [ 0  0| 1]    [   0    0    1]   [ 0  0| 1]

      [ -1   0|1/2]                                                  [ -1   0
      [  0   1|  0]    [           -1             0 2*a0 + 1/2*x0]   [  0   1
a

In [16]:
G, P, _, _ = build_group(7)
P

[
[1 0]  [-1  0]  [-1  0]  [ 1  0]
[0 1], [ 0 -1], [ 0  1], [ 0 -1]
]

In [33]:
PG = MatrixGroup(P).as_permutation_group()
PG

Permutation Group with generators [(), (1,2)(3,4), (1,2), (3,4)]

In [49]:
x = MatrixGroup(P)
x

Matrix group over Rational Field with 4 generators (
[1 0]  [-1  0]  [-1  0]  [ 1  0]
[0 1], [ 0 -1], [ 0  1], [ 0 -1]
)

In [76]:
normalizers(7, ignore_trivial=False)

[
[x0  0]  [ 0 x0]  [x1  0]  [ 0  0]  [ 0  0]  [ 0 x1]
[ 0  0], [ 0  0], [ 0 x0], [x0  0], [ 0 x0], [x0  0]
]

In [44]:

for el in gap(MatrixGroup(P)).AutomorphismGroup().AsList(): 
    print(el)

TypeError: gap() takes exactly 1 positional argument (2 given)